In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

year = 2024

In [2]:
format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}','profit':'{:,.2f}',
              'pct':'{:,.2f}%','net':'{:,.2f}',
              'cost_amt':'{:,.2f}','unit_cost':'{:,.2f}','avg_cost':'{:,.2f}'}

pd.set_option('display.float_format','{:,.2f}'.format)

### Record selection for transactions

In [3]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month,  
             YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
sells.price AS sell_price, buys.price AS buy_price, 
(sells.price - buys.price) AS diff, qty, 
(sells.price * qty) AS sell_amt,
(buys.price * qty) AS buy_amt,
(sells.price - buys.price) * qty AS gross, 
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s
ORDER BY sell_month DESC, name'''

sql = sql % year
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month,  
             YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
sells.price AS sell_price, buys.price AS buy_price, 
(sells.price - buys.price) AS diff, qty, 
(sells.price * qty) AS sell_amt,
(buys.price * qty) AS buy_amt,
(sells.price - buys.price) * qty AS gross, 
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2024
ORDER BY sell_month DESC, name


In [4]:
sells_df.dtypes

name           object
sell_year       int64
sell_month      int64
buy_year        int64
buy_month       int64
sell_price    float64
buy_price     float64
diff          float64
qty             int64
sell_amt      float64
buy_amt       float64
gross         float64
pct           float64
profit        float64
kind           object
dtype: object

In [5]:
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).gross.sum()

sell_year  sell_month  buy_year  buy_month  name 
2024       1           2021      9          BCH     2,400.00
                       2023      9          STA     2,250.00
                                 10         TFFIF   1,000.00
                                 12         TFFIF     875.00
                       2024      1          CRC       150.00
           2           2024      1          BEM       150.00
           3           2024      2          DIF      -150.00
           6           2024      6          JMT     1,260.00
           8           2024      5          WHART   7,500.00
                                 6          JMT     6,120.00
                                            WHART   7,500.00
                                 7          JMT     5,100.00
Name: gross, dtype: float64

In [6]:
sold_grp = sells_df.groupby(['name'])
sold_stocks = sold_grp['sell_amt','buy_amt','qty','gross'].sum()
sold_stocks.sort_values(['name'],ascending=[True]).style.format(format_dict)

C:\Users\User\AppData\Local\Temp\ipykernel_14360\2967423552.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sold_stocks = sold_grp['sell_amt','buy_amt','qty','gross'].sum()


,sell_amt,buy_amt,qty,gross
name,,,,
BCH,"45,800.00","43,400.00","2,000","2,400.00"
BEM,"22,650.00","22,500.00","3,000",150.00
CRC,"21,150.00","21,000.00",600,150.00
DIF,"23,700.00","23,850.00","3,000",-150.00
JMT,"78,150.00","65,670.00","5,100","12,480.00"
STA,"42,250.00","40,000.00","2,500","2,250.00"
TFFIF,"34,250.00","32,375.00","5,000","1,875.00"
WHART,"102,000.00","87,000.00","10,000","15,000.00"


In [7]:
sold_stocks['sell_price'] = sold_stocks['sell_amt'] / sold_stocks['qty']
sold_stocks['buy_price'] = sold_stocks['buy_amt'] / sold_stocks['qty']
cols = 'sell_amt buy_amt gross qty sell_price buy_price'.split()
sold_stocks[cols].sort_values(['name'],ascending=[True]).style.format(format_dict)

,sell_amt,buy_amt,gross,qty,sell_price,buy_price
name,,,,,,
BCH,"45,800.00","43,400.00","2,400.00","2,000",22.90,21.70
BEM,"22,650.00","22,500.00",150.00,"3,000",7.55,7.50
CRC,"21,150.00","21,000.00",150.00,600,35.25,35.00
DIF,"23,700.00","23,850.00",-150.00,"3,000",7.90,7.95
JMT,"78,150.00","65,670.00","12,480.00","5,100",15.32,12.88
STA,"42,250.00","40,000.00","2,250.00","2,500",16.90,16.00
TFFIF,"34,250.00","32,375.00","1,875.00","5,000",6.85,6.47
WHART,"102,000.00","87,000.00","15,000.00","10,000",10.20,8.70


In [8]:
sold_stocks[cols].sort_values(['gross'],ascending=[False]).style.format(format_dict)

,sell_amt,buy_amt,gross,qty,sell_price,buy_price
name,,,,,,
WHART,"102,000.00","87,000.00","15,000.00","10,000",10.20,8.70
JMT,"78,150.00","65,670.00","12,480.00","5,100",15.32,12.88
BCH,"45,800.00","43,400.00","2,400.00","2,000",22.90,21.70
STA,"42,250.00","40,000.00","2,250.00","2,500",16.90,16.00
TFFIF,"34,250.00","32,375.00","1,875.00","5,000",6.85,6.47
BEM,"22,650.00","22,500.00",150.00,"3,000",7.55,7.50
CRC,"21,150.00","21,000.00",150.00,600,35.25,35.00
DIF,"23,700.00","23,850.00",-150.00,"3,000",7.90,7.95


### Record selection for active stocks

In [9]:
sql = '''
SELECT name, YEAR(buys.date) AS year, MONTH(buys.date) AS month,
buys.price AS unit_cost, qty, 
(buys.price * qty) AS cost_amt
FROM buys
JOIN stocks ON buys.stock_id = stocks.id
WHERE status = 'Active'
ORDER BY name, buys.date'''
buys_df = pd.read_sql(sql, conpf)
buys_df.style.format(format_dict)

,name,year,month,unit_cost,qty,cost_amt
0,3BBIF,2018,5,10.00,"30,000","300,000.00"
1,3BBIF,2019,11,10.00,"30,000","300,000.00"
2,3BBIF,2020,2,10.00,"10,000","100,000.00"
3,3BBIF,2020,11,10.00,"30,000","300,000.00"
4,3BBIF,2022,5,11.00,"10,000","110,000.00"
5,3BBIF,2022,6,9.75,"10,000","97,500.00"
6,3BBIF,2022,7,9.15,"10,000","91,500.00"
7,AH,2023,6,37.00,"1,200","44,400.00"
8,AIMIRT,2023,8,11.00,"10,000","110,000.00"
9,AIMIRT,2024,6,10.50,"2,500","26,250.00"


In [10]:
buys_df.groupby(['year','month','name']).cost_amt.sum()

year  month  name  
2016  9      MCS      167,000.00
      10     MCS      334,000.00
      11     MCS      334,000.00
2017  4      MCS      167,000.00
2018  5      3BBIF    300,000.00
                         ...    
2023  9      TOA       26,000.00
2024  2      GVREIT    36,000.00
      6      AIMIRT    26,250.00
             IVL       16,000.00
      7      ORI       13,800.00
Name: cost_amt, Length: 85, dtype: float64

In [11]:
buys_df.groupby(['year','month']).cost_amt.sum()

year  month
2016  9       167,000.00
      10      334,000.00
      11      334,000.00
2017  4       167,000.00
2018  5       300,000.00
2019  11      300,000.00
2020  2       100,000.00
      11      300,000.00
2021  2       147,000.00
      3       520,950.00
      4        66,750.00
      6       200,000.00
      8       163,500.00
      9       396,850.00
      10      472,900.00
      11       44,700.00
      12      486,500.00
2022  1       585,800.00
      2       861,250.00
      3       529,750.00
      4       953,000.00
      5       298,400.00
      6       300,450.00
      7        91,500.00
      8       294,100.00
      9        95,000.00
      10      169,750.00
      11      372,000.00
2023  1       888,100.00
      2       542,850.00
      3       383,400.00
      5        22,800.00
      6        89,040.00
      7       215,040.00
      8       176,000.00
      9       188,600.00
2024  2        36,000.00
      6        42,250.00
      7        13,800.00
Name: cost_am

In [12]:
buys_df.cost_amt.sum()

11650030.0

In [13]:
buys_df.groupby(['name'])['cost_amt','qty'].sum().style.format(format_dict)

C:\Users\User\AppData\Local\Temp\ipykernel_14360\3973236850.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  buys_df.groupby(['name'])['cost_amt','qty'].sum().style.format(format_dict)


,cost_amt,qty
name,,
3BBIF,"1,299,000.00","130,000"
AH,"44,400.00","1,200"
AIMIRT,"136,250.00","12,500"
ASK,"139,500.00","4,500"
ASP,"114,000.00","30,000"
AWC,"44,640.00","9,000"
BCH,"86,800.00","4,000"
CPNREIT,"1,053,000.00","60,000"
DIF,"441,000.00","30,000"


In [14]:
buys_grp = buys_df.groupby(by=['name'])
dtd_stocks = buys_grp['cost_amt','qty'].sum()
dtd_stocks['avg_cost'] = dtd_stocks['cost_amt'] / dtd_stocks['qty']
dtd_stocks.sort_values(['name'],ascending=[True]).style.format(format_dict)

C:\Users\User\AppData\Local\Temp\ipykernel_14360\726847341.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dtd_stocks = buys_grp['cost_amt','qty'].sum()


,cost_amt,qty,avg_cost
name,,,
3BBIF,"1,299,000.00","130,000",9.99
AH,"44,400.00","1,200",37.00
AIMIRT,"136,250.00","12,500",10.90
ASK,"139,500.00","4,500",31.00
ASP,"114,000.00","30,000",3.80
AWC,"44,640.00","9,000",4.96
BCH,"86,800.00","4,000",21.70
CPNREIT,"1,053,000.00","60,000",17.55
DIF,"441,000.00","30,000",14.70


In [15]:
file_name = 'unit-cost.csv'
data_file = data_path + file_name
csv_file = csv_path + file_name
box_file = box_path + file_name
data_file, csv_file, box_file

('../data/unit-cost.csv',
 '\\Users\\User\\iCloudDrive\\unit-cost.csv',
 '\\Users\\User\\Dropbox\\unit-cost.csv')

In [16]:
dtd_stocks.sort_values(['name'],ascending=[True]).to_csv(csv_file)
dtd_stocks.sort_values(['name'],ascending=[True]).to_csv(box_file)
dtd_stocks.sort_values(['name'],ascending=[True]).to_csv(data_file)

### Extra addition

In [17]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, sells.date AS sell_date,
qty, sells.price AS sell_price, buys.price AS buy_price,
sells.net AS sell_amt, buys.net AS buy_amt, profit,
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, sells.id AS sell_id, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s
ORDER BY sells.date, name'''

sql = sql % year
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, sells.date AS sell_date,
qty, sells.price AS sell_price, buys.price AS buy_price,
sells.net AS sell_amt, buys.net AS buy_amt, profit,
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, sells.id AS sell_id, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2024
ORDER BY sells.date, name


In [18]:
df_sells = pd.read_sql(sql, conpf)
df_sells.head()

,name,sell_year,sell_month,sell_date,qty,sell_price,buy_price,sell_amt,buy_amt,profit,pct,sell_id,kind
0,TFFIF,2024,1,2024-01-02,2500,6.70,6.35,"16,712.90","15,910.16",802.74,5.51,1060,DOS
1,STA,2024,1,2024-01-03,2500,16.90,16.00,"42,156.42","40,088.60","2,067.82",5.63,1061,DTD
2,BCH,2024,1,2024-01-04,2000,22.90,21.70,"45,698.55","43,496.13","2,202.42",5.53,1064,DTD
3,TFFIF,2024,1,2024-01-04,2500,7.00,6.60,"17,461.23","16,536.55",924.68,6.06,1062,DOS
4,CRC,2024,1,2024-01-24,600,35.25,35.00,"21,103.16","21,046.51",56.65,0.71,1065,DTD


In [19]:
ttl_by_month = df_sells.groupby(['sell_month'])['profit'].sum()
ttl_by_month

sell_month
1    6,054.31
2       49.99
3     -255.32
6    1,146.37
8   25,596.44
Name: profit, dtype: float64

In [20]:
ttl_by_month = df_sells.groupby(['sell_year','sell_month','name'], as_index=True).agg(
    {
        'profit':['sum','count'],
    }
)
ttl_by_month

profit      
                                 sum count
sell_year sell_month name                 
2024      1          BCH    2,202.42     1
                     CRC       56.65     1
                     STA    2,067.82     1
                     TFFIF  1,727.42     2
          2          BEM       49.99     1
          3          DIF     -255.32     1
          6          JMT    1,146.37     1
          8          JMT   11,015.07     2
                     WHART 14,581.37     2

In [21]:
pd.set_option('max_rows',None)
ttl_by_month.get('profit')

sum  count
sell_year sell_month name                  
2024      1          BCH    2,202.42      1
                     CRC       56.65      1
                     STA    2,067.82      1
                     TFFIF  1,727.42      2
          2          BEM       49.99      1
          3          DIF     -255.32      1
          6          JMT    1,146.37      1
          8          JMT   11,015.07      2
                     WHART 14,581.37      2

In [22]:
ttl_by_month.groupby(level='name').sum()

profit      
            sum count
name                 
BCH    2,202.42     1
BEM       49.99     1
CRC       56.65     1
DIF     -255.32     1
JMT   12,161.44     3
STA    2,067.82     1
TFFIF  1,727.42     2
WHART 14,581.37     2

In [23]:
ttl_by_month.sum()

profit  sum     32,591.79
        count       12.00
dtype: float64

### Profit by Stock

In [24]:
name = 'JMT'
sql = '''
SELECT name, sells.date AS sell_date,
qty, sells.price AS sell_price, buys.price AS buy_price,
sells.net AS sell_amt, buys.net AS buy_amt, profit,
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, sells.id AS sell_id, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s AND name = '%s'
ORDER BY sells.date DESC'''

sql = sql % (year, name)
print(sql)

df_name = pd.read_sql(sql, conpf)
df_name.style.format(format_dict)



SELECT name, sells.date AS sell_date,
qty, sells.price AS sell_price, buys.price AS buy_price,
sells.net AS sell_amt, buys.net AS buy_amt, profit,
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, sells.id AS sell_id, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2024 AND name = 'JMT'
ORDER BY sells.date DESC


,name,sell_date,qty,sell_price,buy_price,sell_amt,buy_amt,profit,pct,sell_id,kind
0,JMT,2024-08-27,"1,800",16.40,13.00,"29,454.61","23,451.83","6,002.78",26.15%,1073,DTD
1,JMT,2024-08-22,"1,500",14.90,11.50,"22,300.50","17,288.21","5,012.29",29.57%,1071,HD
2,JMT,2024-06-05,"1,800",14.60,13.90,"26,221.79","25,075.42","1,146.37",5.04%,1069,DTD


In [25]:
profit = df_name.profit.sum()
cost = df_name.buy_amt.sum()
percent = round(profit/cost*100,2)
profit, percent

(12161.439999999999, 18.48)

In [26]:
df_by_price = df_name.groupby(['sell_price'], as_index=True).agg(
    {
        'sell_price':['count'],
    }
)
df_by_price

,sell_price
,count
sell_price,
14.60,1
14.90,1
16.40,1


In [27]:
df_by_price = df_name.groupby(['sell_price','buy_price'], as_index=True).agg(
    {
        'sell_price':['count'],
    }
)
df_by_price

,,sell_price
,,count
sell_price,buy_price,
14.60,13.90,1
14.90,11.50,1
16.40,13.00,1
